In [1]:
import os
import numpy as np
from bct import modularity

from signed_modularity import modularity_louvain_und_sign

In [2]:
def load_group_files(base_url, group_path):
    path = '{}/{}/'.format(base_url, group_path)
    file_names = sorted(os.listdir(path))
    files = ['{}{}'.format(path, elem) for elem in file_names]
    return files

def load_matrix_subjects(base_url, group_path):
    group_files = load_group_files(base_url, group_path)
    return _load_matrix_subject_with_files(group_files)

def _load_matrix_subject_with_files(files):
    subjects = []
    for file in files:
        mat = np.loadtxt(file, delimiter=' ')
        np.fill_diagonal(mat, 0)
        subjects.append(mat)
    return np.array(subjects)

def nearest_neighbour_edge_centrality(edge, matrix):
    degree_a = np.sum(matrix[edge[0],])
    degree_b = np.sum(matrix[edge[1],])
    w_ab = matrix[edge[0], edge[1]]

    return (degree_a + degree_b - 2 * w_ab) / (np.abs(degree_a - degree_b) + 1) * w_ab


In [3]:
data_type = 'correlation'
datasets = ['adolescents', 'children', 'eyesclosed', 'male', 'female', 'complete_abide']

n_nodes = 116

In [23]:
idx = 3

In [24]:
dataset = datasets[idx]

diff_mat = np.loadtxt(f"../abide_datasets/{data_type}_diff_mat/{dataset}/td_asd_diff_mat_whole_dataset.txt")

td_subjects = load_matrix_subjects(f'../abide_datasets/{data_type}/{dataset}', 'td')
asd_subjects = load_matrix_subjects(f'../abide_datasets/{data_type}/{dataset}', 'asd')

In [37]:
ci_ret, q = modularity_louvain_und_sign(np.mean(td_subjects, axis=0), qtype='sta')
ci_ret_org, q_org = modularity.modularity_louvain_und_sign(np.mean(td_subjects, axis=0), qtype='sta')

In [38]:
print(ci_ret)

[4 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 1 1 1 1 1 1 1 1 4 4 1 1 4 4 1 1 4
 4 5 5 4 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 3 3 3 3 4 4 1 1 3 1 4 5 4 4 4 4
 4 4 4 4 4 4 4 4 1 4 1 1 1 1 1 1 5 5 5 5 5 5 2 5 2 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5]


In [39]:
print(ci_ret_org)

[3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 1 1 1 1 1 1 1 1 3 3 1 1 3 3 1 1 3
 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 1 2 1 1 3 3 1 1 2 1 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 1 3 1 1 1 1 1 1 4 4 4 4 4 4 2 4 2 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4]


In [40]:
(q, q_org)

(0.5616368048360386, 0.5547785122319459)

In [41]:
ci_ret, q = modularity_louvain_und_sign(np.mean(asd_subjects, axis=0), qtype='sta')
ci_ret_org, q_org = modularity.modularity_louvain_und_sign(np.mean(asd_subjects, axis=0), qtype='sta')

In [42]:
print(ci_ret)

[3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 1 1 1 1 1 1 1 1 3 3 1 1 3 3 1 1 3
 3 2 2 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 4 4 4 4 3 3 1 1 4 1 3 2 3 3 3 3
 3 3 3 3 3 3 3 3 1 3 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2]


In [43]:
print(ci_ret_org)

[3 3 1 1 1 1 1 1 1 1 3 3 3 3 1 1 3 3 3 3 1 1 1 1 1 1 1 1 3 3 1 1 3 3 1 1 3
 3 2 2 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 2 2 3 3 3 3 1 1 2 1 3 2 3 3 3 3
 3 3 3 3 3 3 3 3 1 3 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2]


In [44]:
(q, q_org)

(0.5662440697042003, 0.5721015109853619)